In [39]:
# format: [[word_list], [tag_list]]
# word_list = [[tweet1],[tweet2], ...]
# tag_list = [[tags for tweet1],[tags for tweet2], ...]
# tweet<n> = [word1, word2, ...]
# tags for tweet<n> = [tag1, tag2, ...]
def read_train(file_name):
    in_file = open(file_name,'r',encoding='utf8')
    l = []
    words = []
    tags = []
    word_list = []
    tag_list = []
    for line in in_file:
        x = line.strip().split()
        if x != []:
            words.append(x[0])
            tags.append(x[1].rstrip('\n'))
        else:
            word_list.append(words)
            tag_list.append(tags)
            words = []
            tags = []

    l.append(word_list)
    l.append(tag_list)
    in_file.close()
    return l
    
# reading and writing to files 
# format:[[tweet1],[tweet2], ...]
# tweet<n> = [word1, word2, ...]
def read_dev_in(file_name):
    in_file = open(file_name,'r',encoding='utf8')
    l = []
    tweet = []
    for line in in_file:
        tweet.append(line.strip())
        if line.strip()=="":
            tweet.remove("")
            l.append(tweet)
            tweet=[]
                
        
    in_file.close()
    return l

def write_devp2(language,word_list,tag_list):
    file_name = language+"/"+"dev.p2.out"
    if os.path.isfile(file_name):
        print('file exist')
        try:
            os.remove(file_name)
            print("deleted file")
        except OSError:
            print("ERROR")
            
    out_file = open(file_name,'a',encoding='utf8')

    for i in range(len(word_list)):
        for j in range(len(word_list[i])):
            out_file.write(word_list[i][j]+" "+tag_list[i][j]+"\n")
        out_file.write(" \n")
    
    out_file.close()
        
def write_devp3(language,word_list,tag_list):
    file_name = language+"/"+"dev.p3a.out"
    if os.path.isfile(file_name):
        print('file exist')
        try:
            os.remove(file_name)
            print("deleted file")
        except OSError:
            print ("error")
            #pass
    out_file = open(file_name,'a',encoding='utf8')

    for i in range(len(word_list)): 
        lw = len(word_list[i])
        lt = len(tag_list[i])
        #print("length of words = "+str(lw))
        #print("length of tags = "+str(lt))
        if lt!=lw:
            print(word_list[i])
            print(tag_list[i])
            print (i)
        
        for j in range(len(word_list[i])):
            out_file.write(word_list[i][j]+" "+tag_list[i][j]+"\n")
        out_file.write(" \n")
    
    out_file.close()   
    



In [64]:
import time, os, math
from collections import Counter
from itertools import count

class HMM:

    tags = ["B-negative","B-neutral","B-positive","I-negative","I-neutral","I-positive","O"]
    word_tag_em = {} # {(word,tag): emission probability}
    
    def __init__(self,train_words,train_tags):
        self.train_words = train_words 
        self.train_tags = train_tags 
        
        self.join_train_words = [j for i in train_words for j in i] #convert nested lists into one big list for zipping
        self.join_train_tags = [j for i in train_tags for j in i]
        
        self.tag_count = Counter(self.join_train_tags) # total count for each tag found in training data
        self.word_tag_count = Counter(list(zip(self.join_train_words,self.join_train_tags))) # total count for each word-tag pair found in training data
        
        self.new_word = self.new_word_emission() # (tag,emission)
        self.train_emission()
        
        #print(self.tag_count)
        #print(self.word_tag_count)
        #print(self.new_word)
        #print(self.word_tag_em)
#=========================== PART 2 =============================
    #counting number of occurence of y_i
    def count_tag(self,tag):                
        return self.tag_count[tag]
    
    
    #2a count(y pair x)/ count(y)
    def emission(self,word,tag):
        return self.word_tag_count[(word,tag)]
    
    
    #2b emission count(y pair x)/ (count(y)+1)
    def get_emission_prob(self,word,tag):
        return self.word_tag_count[(word,tag)]/(self.count_tag(tag)+1)
    
    #2b finding tag for new word using estimation 1/ (count(y)+1)   
    def new_word_emission(self):
        em=[]
        for tag in self.tags:  
            em.append(1/(self.count_tag(tag)+1))
        #return label using the index of maximum emission found
        new_word_em = max(em)
        new_word_tag = self.tags[em.index(max(em))]
        return (new_word_tag,new_word_em)
    
    def train_emission(self):
        #for each (word,tag) pair, calculate emission score and put in dictionary
        for wt_pair in self.word_tag_count:
            self.word_tag_em[wt_pair]=self.get_emission_prob(wt_pair[0],wt_pair[1])
        return self.word_tag_em
    
    def sentiment_analysis(self,test_data):
        
        predicted_tags = []
        for tweet in test_data:
            max_tag = []
            for word in tweet:
                temp = []
                if word in self.join_train_words:
                    #print("\nI AM INSIDE\n")
                    for tag in self.tags:
                        if (word,tag) in self.word_tag_em:
                            temp.append((self.word_tag_em[(word,tag)]))
                        else:
                            temp.append(0)
                    #print(temp)
                    max_em = max(temp)
                    p_tag = self.tags[temp.index(max(temp))]

                else:
                    #print("\nI AM OUTSIDE\n")
                    p_tag = self.new_word[0]
                    
                if p_tag[0] == "I":
                    
                    if len(max_tag) == 0 or max_tag[-1]=="O" or max_tag[-1][1:]!=p_tag[1:]:                           
                        p_tag = "B"+p_tag[1:]
                        
                max_tag.append(p_tag)
                
            predicted_tags.append(max_tag)
        return predicted_tags
    
    
                        

                    


In [65]:
train_data = read_train("EN/train")
test_data = read_dev_in("EN/dev.in")
hmm = HMM(train_data[0],train_data[1])
starttime = time.time()
predicted_tags = hmm.sentiment_analysis(test_data)
write_devp2("EN",test_data,predicted_tags)
#print(test_data)
elapsed = time.time()-starttime

print ("time taken = "+str(elapsed)+"s")

file exist
deleted file
time taken = 1.5312809944152832s
